In [1]:
pip install selenium
pip install webdriver-manager
apt-get update -q
apt-get install -y wget curl unzip
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
apt --fix-broken install -y
google-chrome-stable --version
apt-get install -y chromium-chromedriver
cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 17.1 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [2]:
!apt update
!apt install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,211 B]
Hit:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 3,036 B in 3s (1,076 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
55

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Lista różnych wartości wyszukiwania
search_terms = ['fabric'] #, 'synapse', 'machine', 'lakehouse', 'architecture', 'engineer']

# Globalna lista do przechowywania wyników
all_events = []

# Ustawienia przeglądarki (przykład dla Chrome)
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--headless")  # Uncomment this line if you want to run it in the background
options.add_argument("window-size=1920,1080")
options.add_argument("--remote-debugging-port=9222")  # Added for stability in headless mode
options.headless = False  # Set to False to see the browser window


# Specify the path to your Chrome binary
options.binary_location = "/content/chromedriver.exe"  # Update this path

# Set up the driver with the correct Service object
service = Service(ChromeDriverManager().install())  # ChromeDriverManager handles downloading the driver

# Initialize the webdriver
driver = webdriver.Chrome( options=options)
# Funkcja do ładowania wszystkich wydarzeń
def load_all_events(url):
    driver.get(url)

    # Czekamy na załadowanie przycisku 'Show More Events' i klikamy go
    while True:
        try:
            # Czekamy, aż przycisk będzie dostępny
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "loadresults-button"))
            )

            # Jeśli przycisk jest widoczny, klikamy go za pomocą JavaScript
            driver.execute_script("arguments[0].click();", load_more_button)

            time.sleep(15)  # Czekamy chwilę na załadowanie nowych wyników

            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.title-date'))
            )

            # Wykonanie JavaScriptu na stronie, aby upewnić się, że dane zostały załadowane
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Poczekaj na załadowanie danych

        except Exception:
            print("Nie ma już więcej przycisków 'Show More Events'.")
            break

# Funkcja do pobierania wydarzeń
def fetch_events(search_term):
    # URL strony z parametrem search
    url = f'https://events.microsoft.com/en-us/allevents/?language=English&language=Polski&clientTimeZone=1&search={search_term}'

    # Załaduj wszystkie wydarzenia
    load_all_events(url)

    # Pobieranie strony po załadowaniu wszystkich wydarzeń
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Sprawdź całą stronę, aby sprawdzić, jak wygląda struktura
    #print(soup.prettify())  # To wydrukuje całą strukturę HTML strony

    # Znalezienie wszystkich elementów z wydarzeniami
    events = soup.find_all('div', class_='c-card bgcolor-white grideventscroll')

    # Iteracja po znalezionych wydarzeniach
    for event in events:
        title = event.find('h3', class_='c-heading-6').text.strip() if event.find('h3', class_='c-heading-6') else 'Brak tytułu'

        # Pobieranie linku do rejestracji
        registration_button = event.find('button', class_='c-button', attrs={'onclick': True})
        if registration_button:
            onclick_value = registration_button.get('onclick')
            registration_link = onclick_value.split("window.open('")[1].split("', '_blank'")[0]
        else:
            registration_link = 'Brak linku'

        # Pobieranie daty i czasu z linku rejestracji
        date_time = get_date_from_registration_link(registration_link)

        # Pobieranie opisu z linku rejestracji
        description_long = get_text_from_registration_link(registration_link)


        # Pobieranie formatu wydarzenia
        format_element = event.find('p', class_='digital')
        event_format = format_element.text.strip() if format_element else 'Brak formatu'

        # Pobieranie języka wydarzenia
        language_element = event.find('p', class_='gridcard-deliverylan-min')
        language = language_element.text.strip() if language_element else 'Brak języka'

        # Pobieranie typu wydarzenia
        event_type_element = event.find('p', class_='card-eventType')
        event_type = event_type_element.text.strip() if event_type_element else 'Brak typu'

        # Pobieranie opisu wydarzenia
        description_element = event.find('p', class_='gridcard-description-min')
        description = description_element.text.strip() if description_element else 'Brak opisu'

        # Dodajemy dane do listy `all_events`
        all_events.append({
            'title': title,
            'date_time': date_time,
            'description_long': description_long,
            'registration_link': registration_link,
            'event_format': event_format,
            'language': language,
            'event_type': event_type,
            'description': description,
            'search_term': search_term
        })

# Funkcja do wyciągania daty z linku rejestracji
def get_date_from_registration_link(registration_link):
    driver.get(registration_link)

    try:
        # Czekamy na załadowanie elementu z datą
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#banner > div > div > div > div.col.wrapperColumn > div > div.cbdatewrapper > div.cbdate > span:nth-child(2)"))
        )

        # Pobieranie daty z wybranego elementu
        date_element = driver.find_element(By.CSS_SELECTOR, "#banner > div > div > div > div.col.wrapperColumn > div > div.cbdatewrapper > div.cbdate > span:nth-child(2)")
        date_time = date_element.text.strip() if date_element else 'Brak daty'
        return date_time
    except Exception as e:
        #print(f"Nie udało się pobrać daty z {registration_link}: {e}")
        return 'Brak daty'

# Funkcja do wyciągania opisu z linku rejestracji
def get_text_from_registration_link(registration_link):
    driver.get(registration_link)

    try:
        # Czekamy na załadowanie elementu z datą
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,"#main > app-tc-registrationpage > div > div > div > div > div:nth-child(2) > div.col-md-7.noprgutters > app-tc-agenda > div > div.note > div"))
        )

        # Pobieranie daty z wybranego elementu
        text_element = driver.find_element(By.CSS_SELECTOR,"#main > app-tc-registrationpage > div > div > div > div > div:nth-child(2) > div.col-md-7.noprgutters > app-tc-agenda > div > div.note > div")
        text_desc = text_element.text.strip() if text_element else 'Brak opisu'
        return text_desc
    except Exception as e:
        #print(f"Nie udało się pobrać tekstu z {registration_link}: {e}")
        return 'Brak opisu'

# Iteracja po wszystkich terminach wyszukiwania
for term in search_terms:
    print(f"\nWyniki wyszukiwania dla: {term}")
    fetch_events(term)

# Wyświetlanie wszystkich zebranych wyników
print("\nWszystkie zebrane wyniki:")
"""
for event in all_events:
    print(f'Tytuł: {event["title"]}')
    print(f'Data i czas: {event["date_time"]}')
    print(f'Link do rejestracji: {event["registration_link"]}')
    print(f'Format wydarzenia: {event["event_format"]}')
    print(f'Język: {event["language"]}')
    print(f'Typ wydarzenia: {event["event_type"]}')
    print(f'Opis: {event["description"]}')
    print(f'Wyszukiwanie: {event["search_term"]}')
    print('---')
"""

# Zamknięcie przeglądarki
driver.quit()
df = pd.DataFrame(all_events)
df.head(15)

: 

In [7]:
df.to_excel('events.xlsx', index=True)
print("DataFrame zapisany do pliku output.xlsx")

DataFrame zapisany do pliku output.xlsx


In [11]:

!pip install qdrant_client
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.4/306.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.


In [12]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [13]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [15]:
qdrant.recreate_collection(
    collection_name="",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-15-0bb15112fa33>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [23]:
data = df['description_long'].copy().to_string()

print(data)

0      Fabric Analyst in a Day - intecs data, s.r.o. ...
1      Fabric Analyst in a Day (FAIAD) - UB Technolog...
2      Fabric Analyst in a Day - Intellify Solutions ...
3      Fabric Analyst in a Day (FAIAD) - Lantern Stud...
4      Fabric Analyst in a Day - Data Integrity Servi...
5      Adapt your skills and master the tools you’ll ...
6      Simplifying ESG reporting with Microsoft Fabri...
7      Fabric Analyst in a Day (FAIAD) - MAQ Software...
8      Localized times:\nGreenwich Mean Time: 2:00PM ...
9      Fabric Analyst in a Day (FAIAD) - Thorogood As...
10     Fabric Analyst in a Day - Intellify Solutions ...
11     In Person: Fabric Analyst in a Day - Axians - ...
12     Join us for an exclusive webinar on the future...
13     Fabric Analyst in a Day (FAIAD) - Calian Digit...
14     Fabric Analyst in a Day - Data Integrity Servi...
15                                            Brak opisu
16     Fabric Analyst in a Day - ROYAL CYBER INC - Un...
17     Fabric Analyst in a Day 

In [24]:
#data = df['description_long'].copy()

qdrant.upload_records(
    collection_name="",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc['description_long']),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

TypeError: string indices must be integers, not 'str'

In [ ]:
hits = qdrant.search(
    collection_name="",
    query_vector=encoder.encode("your query here...").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

In [ ]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [ ]:
# Now time to connect to the large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "if-local-sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "Define the role for LLM..."},
        {"role": "user", "content": "Put your query here...."},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)